In [1]:
import sys
sys.path.append('../')

from envs.GridMaps import *
from utilities.transitionDatasetGeneration import *

In [2]:
shape = "14x14"                     #Tamanho del grid '5x5' o '14x14'
store = True                        #'True' para guardar el ensemble generado en la carpeta

if shape == "5x5":
    maze = maze5x5["maze"]          #Nombre del mapa que se va a utilizar
    n_epochs = 15000                #Numero de epocas a entrenar cada modelo
    folder_name = "5x5_5_model"     #Nombre de la carpeta donde se guardan los modelos
    n_models = 5                    #Numero de modelos en el ensemble
    hidden_size = 32                #Numero de neuronas en las capas ocultas de los modelos

elif shape == "14x14":              
    maze = maze14x14["maze"]        #Nombre del mapa que se va a utilizar
    n_epochs = 120000               #Numero de epocas a entrenar cada modelo
    folder_name = "5_model"         #Nombre de la carpeta donde se guardan los modelos
    n_models = 5                    #Numero de modelos en el ensemble
    hidden_size = 512               #Numero de neuronas en las capas ocultas de los modelos
    
#Generacion del dataset offline
train_transitions, val_transitions = transition_generator(maze)

In [3]:
models_arr = []

for i in range(n_models):
    print(f"Training model {i+1}...")

    model = ModelTrainer(train_transitions, val_transitions, n_epochs, hidden_size, rand_seed=True)

    model.fit()
    models_arr.append(model)

Training model 1...


100%|██████████| 120000/120000 [52:42<00:00, 37.94it/s] 


Trained successfully!
Final loss: 7.184115474956343e-06

Training model 2...


100%|██████████| 120000/120000 [47:32<00:00, 42.07it/s]


Trained successfully!
Final loss: 6.542316259583458e-05

Training model 3...


...

Trained successfully!
Final loss: 4.470402927836403e-05

Training model 4...


100%|██████████| 120000/120000 [47:29<00:00, 42.11it/s]


Trained successfully!
Final loss: 6.841804861323908e-05

Training model 5...


100%|██████████| 120000/120000 [47:45<00:00, 41.88it/s]

Trained successfully!
Final loss: 1.0458199540153146e-05



In [4]:
if store: store_models(models_arr, folder_name)

In [5]:
print('\nEVALUATION:\n')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

arr1 = []
arr1_raw = []
arr2 = []
arr2_raw = []

for model in models_arr:
    # Single predictions
    with torch.no_grad():
        # Move the input data to the GPU
        inp1 = torch.FloatTensor([[5, 3, 3]]).to(device)
        inp2 = torch.FloatTensor([[5, 4, 3]]).to(device)
        
        # Perform inference on the GPU
        resultado1 = model.model(inp1)
        resultado2 = model.model(inp2)
        
        # Move the result back to CPU and convert to numpy
        resultado1 = resultado1.cpu().detach().numpy()
        resultado2 = resultado2.cpu().detach().numpy()

        arr1.append(np.int32(np.round(resultado1[0])))
        arr1_raw.append(resultado1[0])
        arr2.append(np.int32(np.round(resultado2[0])))
        arr2_raw.append(resultado2[0])
        
for elem, elem2 in zip(arr1, arr2):
    print(elem, '         ', elem2)

#whats the mean and the std of the raw values?
print('\n')
print('Mean of raw values:')
print(np.mean(arr1_raw, axis=0))
print(np.mean(arr2_raw, axis=0))
print('\n')
print('Std of raw values:')
print(np.std(arr1_raw, axis=0), '     ', np.linalg.norm(np.std(arr1_raw, axis=0)))
print(np.std(arr2_raw, axis=0), '     ', np.linalg.norm(np.std(arr2_raw, axis=0)))



EVALUATION:

Using device: cuda
[5 4]           [5 5]
[5 4]           [5 5]
[5 4]           [5 5]
[5 4]           [5 5]
[5 4]           [5 5]


Mean of raw values:
[4.9993773 3.9983373]
[5.0009923 4.9991593]


Std of raw values:
[0.00542308 0.00251136]       0.0059763477
[0.00379623 0.00423822]       0.005689801
